In [41]:
# Install dependencies
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import json


In [38]:
#URL's of pages to be scraped
nasa_news_url = "https://mars.nasa.gov/news/"
nasa_news_al = "https://mars.nasa.gov/api/v1/news_items/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
jpl_images_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
mars_weather_url = "https://twitter.com/marswxreport?lang=en"
mars_facts_url = "http://space-facts.com/mars/"
usgs_images_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"


In [48]:
#Mars news scrape
# response = requests.get(nasa_news_url)
# soup = BeautifulSoup(response.text, "html.parser")
# print(soup.prettify())

In [73]:
data = requests.get(nasa_news_al).json()["items"]
response = requests.get(nasa_news_al).json()
# title = soup.find("title")

In [47]:
with open("data_file.json", "w+") as write_file:
    json.dump(data, write_file)

In [68]:
pprint(response)

{'items': [{'body': '<hr />\n'
                    "<p><i>A batch of new papers summarizes the lander's "
                    'findings above and below the surface of the Red '
                    'Planet.</i></p>\n'
                    '\n'
                    '<hr />\n'
                    '<p>A new understanding of Mars is beginning to emerge, '
                    "thanks to the first year of NASA's InSight lander "
                    'mission. Findings described in a set of six papers '
                    'published today reveal a planet alive with quakes, dust '
                    'devils and strange magnetic pulses.</p>\n'
                    '\n'
                    '<p><a '
                    'href="/insight/mission/science/research-papers/">Five of '
                    'the papers</a> were published in Nature. An additional '
                    'paper in Nature Geoscience details the InSight '
                    "spacecraft's landing site, a shallow crater nicknamed "


                    'path.</p>\r\n'
                    '\r\n'
                    '<p>"We are preparing to achieve pivotal milestones this '
                    'year in development of the Space Launch System rocket, '
                    'Orion spacecraft and the Gateway. These make up the '
                    'backbone of our Artemis program and are fully supported '
                    'by this budget. They constitute our ability to build a '
                    'sustainable lunar presence and eventually send human '
                    'missions to Mars.</p>\r\n'
                    '\r\n'
                    '<p>"Most noteworthy, is the President\'s direct funding '
                    'of more than $3 billion for the development of a human '
                    'landing system. This is the first time we have had direct '
                    'funding for a human lander since the Apollo Program. We '
                    "are serious about our 2024 goals, and the President's "
   

                    '<p>A compilation of Mars 2020 construction video and '
                    'mission animation is available at:</p>\r\n'
                    '\r\n'
                    '<p style="text-align:center"><a '
                    'href="http://bit.ly/mars2020reel">http://bit.ly/mars2020reel</a></p>\r\n'
                    '\r\n'
                    '<p>For more information about the mission, go to:</p>\r\n'
                    '\r\n'
                    '<p style="text-align:center"><a '
                    'href="https://mars.nasa.gov/mars2020/">mars.nasa.gov/mars2020/</a></p>\r\n'
                    '\r\n'
                    '<p><b>News Media Contacts</b></p>\r\n'
                    '\r\n'
                    '<p>DC Agle<br />\r\n'
                    'Jet Propulsion Laboratory, Pasadena, Calif.<br />\r\n'
                    '818-393-9011<br />\r\n'
                    '<a '
                    'href="mailto:agle@jpl.nasa.gov">agle@jpl.nasa.gov</a></p>\r\n'
        

                    'extended hydrogen atmosphere, and the absence of a global '
                    'dipole magnetic field) are more commonly available at '
                    'Mars than those needed to create other types of '
                    'aurora,&rdquo; said Hughes. &ldquo;Also, the connection '
                    'between MAVEN&rsquo;s observations of increased '
                    'atmospheric escape and increases in proton aurora '
                    'frequency and intensity means that proton aurora can '
                    'actually be used as a proxy for what&rsquo;s happening in '
                    'the hydrogen corona surrounding Mars, and therefore, a '
                    'proxy for times of increased atmospheric escape and water '
                    'loss.&rdquo; </p>\n'
                    '\n'
                    "<p>This research was funded by the MAVEN mission. MAVEN's "
                    'principal investigator is based at the University of '
        

                    '</figcaption>\n'
                    '</figure>\n'
                    '</div>\r\n'
                    '\r\n'
                    '<p>To demonstrate a precision landing, Apollo 12 mission '
                    'planners could have chosen just about anywhere on the '
                    'nearside of the Moon by targeting any of literally '
                    'millions of known geologic features. In the end, they '
                    'chose for Pete and Al a relatively nondescript crater in '
                    'the Ocean of Storms because JPL had plunked down a '
                    'spacecraft there two-and-a-half years earlier.</p>\r\n'
                    '\r\n'
                    '<p>"When Pete and Al put the lunar module Intrepid down '
                    'within about 520 feet [160 meters] of Surveyor 3, it gave '
                    'NASA the confidence to later send Apollo 15 to Hadley '
                    'Rille, Apollo 16 to go to the Descartes High

            'missions': [[175, 'mep', 'Mars Exploration Program'],
                         [176, 'msl', 'Mars Science Laboratory']],
            'news_type': [[165, 'feature', 'Feature Stories'],
                          [166, 'latest_findings', 'Latest Findings']],
            'status': 1,
            'target': '_self',
            'thumb': '/system/news_items/list_view_images/8548_mars_seasonal_oxygen_gale_crater-320x240.jpg',
            'title': 'With Mars Methane Mystery Unsolved, Curiosity Serves '
                     'Scientists a New One: Oxygen',
            'updated_at': '2019-11-12T15:45:59.701Z',
            'url': '/news/8548/with-mars-methane-mystery-unsolved-curiosity-serves-scientists-a-new-one-oxygen/'},
           {'body': "<figure class='column-width embedded_video "
                    "video_player_container'>\n"
                    "<div class='video_wrapper'>\n"
                    '<iframe width="1225" height="689" '
                    'src="https://www.yout

                    'Glen Etive 2 into SAM. The portable lab contains 74 small '
                    'cups used for testing samples. Most of the cups function '
                    'as miniature ovens that heat the samples; SAM then '
                    '"sniffs" the gases that bake off, looking for chemicals '
                    'that hold clues about the Martian environment billions of '
                    'years ago, when the planet was friendlier to microbial '
                    'life.</p>\n'
                    '\n'
                    "<p>But nine of SAM's 74 cups are filled with solvents the "
                    'rover can use for special "wet chemistry" experiments. '
                    'These chemicals make it easier for SAM to detect certain '
                    'carbon-based molecules important to the formation of '
                    'life, called organic compounds.</p>\n'
                    '\n'
                    "<p>Because there's a limited number of wet-chem

                    'weeks. The next tactic they\'ll try will be "pinning" the '
                    "mole against the hole it's in. Credits: NASA/JPL-Caltech\n"
                    '</figcaption>\n'
                    '</figure>\n'
                    '\n'
                    '<p>"We\'re going to try pressing the side of the scoop '
                    'against the mole, pinning it to the wall of its hole," '
                    'said InSight Deputy Principal Investigator Sue Smrekar of '
                    "NASA's Jet Propulsion Laboratory in Pasadena, California. "
                    '"This might increase friction enough to keep it moving '
                    'forward when mole hammering resumes."</p>\n'
                    '\n'
                    '<p>Whether the extra pressure on the mole will compensate '
                    'for the unique soil remains an unknown.</p>\n'
                    '\n'
                    '<p>Designed to burrow as much as 16 feet (5 meters) '
     

In [69]:
# https://hackersandslackers.com/extract-data-from-complex-json-python/
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [74]:
titles = extract_values(data,"title")
print(titles)

TypeError: isinstance() arg 2 must be a type or tuple of types